In [19]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
import yaml

In [11]:
# load all results from the target directory and put them in a list 
# of dictionaries
results = []
tgtdir = "logs/benchmarks/ratio_loss/"
# list all files and subdirs in the target directory
files = glob("**/srf_estimates.csv", root_dir=tgtdir, recursive=True)

print(f"Extracted {len(files)} that look like {files[0]}")


Extracted 513 that look like tcga-B/gaussian/000034/ipw_classifier/srf_estimates.csv


In [36]:
configs = {}
results = []

for f in tqdm(files):
    comps = f.split("/")
    expid = "_".join([comps[0], comps[1], comps[2], comps[3]])
    
    # config file
    config_file = f.replace("srf_estimates.csv", "hparams.yaml")
    with open(tgtdir + "/" + config_file, "r") as io:
        configs[expid] = yaml.load(io, Loader=yaml.BaseLoader)
    
    # read pandas
    df = pd.read_csv(tgtdir + "/" + f)
    df["dataset"] = comps[0]
    df["family"] = comps[1]
    df["seed"] = comps[2]
    df["experiment"] = comps[3]
    df["expid"] = expid
    df["setup"] = "_".join([comps[0], comps[1]])

    results.append(df)

results = pd.concat(results)
results.head()

100%|██████████| 513/513 [00:14<00:00, 35.61it/s]


,train_srf,test_srf,train_srf_ipw,test_srf_ipw,train_srf_aipw,test_srf_aipw,train_srf_tr,test_srf_tr,train_srf_outcome,test_srf_outcome,true_srf_train,true_test_srf,fluctuation,dataset,family,seed,experiment,expid,setup
0,-0.017059,-0.034774,-0.017059,-0.034774,-0.017586,-0.037695,-0.099748,-0.099754,-0.099748,-0.099754,0.000000,-0.016654,0.0,tcga-B,gaussian,000034,ipw_classifier,tcga-B_gaussian_000034_ipw_classifier,tcga-B_gaussian
1,0.066279,0.050343,0.066279,0.050343,0.059345,0.043296,-0.091637,-0.091454,-0.091637,-0.091454,0.058705,0.043536,0.0,tcga-B,gaussian,000034,ipw_classifier,tcga-B_gaussian_000034_ipw_classifier,tcga-B_gaussian
2,0.142527,0.124314,0.142527,0.124314,0.128547,0.109999,-0.083727,-0.083368,-0.083727,-0.083368,0.120186,0.106441,0.0,tcga-B,gaussian,000034,ipw_classifier,tcga-B_gaussian_000034_ipw_classifier,tcga-B_gaussian
3,0.218915,0.198503,0.218915,0.198503,0.200071,0.178393,-0.076098,-0.075572,-0.076098,-0.075572,0.184443,0.172061,0.0,tcga-B,gaussian,000034,ipw_classifier,tcga-B_gaussian_000034_ipw_classifier,tcga-B_gaussian
4,0.282403,0.267638,0.282403,0.267638,0.258732,0.242509,-0.068816,-0.068136,-0.068816,-0.068136,0.251477,0.240397,0.0,tcga-B,gaussian,000034,ipw_classifier,tcga-B_gaussian_000034_ipw_classifier,tcga-B_gaussian


In [45]:
train_bias2 = results.groupby(["setup", "experiment"]).apply(lambda x: np.mean((x["train_srf"] - x["true_srf_train"])**2))
train_variance = results.groupby(["setup", "experiment"]).apply(lambda x: np.var(x["train_srf"]))
test_bias2 = results.groupby(["setup", "experiment"]).apply(lambda x: np.mean((x["test_srf"] - x["true_test_srf"])**2))
test_variance = results.groupby(["setup", "experiment"]).apply(lambda x: np.var(x["test_srf"]))
train_mse = train_bias2 + train_variance
test_mse = test_bias2 + test_variance

metrics = pd.concat([train_bias2, train_variance, train_mse, test_bias2, test_variance, test_mse], axis=1)
metrics.columns = ["train_bias2", "train_variance", "train_mse", "test_bias2", "test_variance", "test_mse"]
metrics

train_bias2  train_variance  train_mse   
setup           experiment                                               
ihdp-N_gaussian ipw_classifier     0.038951        0.008052   0.047003  \
                ipw_hybrid         0.033362        0.023069   0.056431   
                ipw_ps             0.044179        0.027284   0.071463   
news-N_gaussian ipw_classifier     0.007586        0.002354   0.009941   
                ipw_hybrid         0.000703        0.005306   0.006008   
                ipw_ps             0.002819        0.005282   0.008101   
sim-B_gaussian  ipw_classifier     0.105004        0.017611   0.122615   
                ipw_hybrid         0.075917        0.092587   0.168505   
                ipw_ps             0.162460        0.258506   0.420966   
tcga-B_gaussian ipw_classifier     0.005199        0.069814   0.075013   
                ipw_hybrid         0.022872        0.145842   0.168714   
                ipw_ps             0.324843        0.661582   0.986425   

                                test_bias2  test_variance  test_mse  
setup           experiment                                           
ihdp-N_gaussian ipw_classifier    0.039298       0.011955  0.051253  
                ipw_hybrid        0.037080       0.025182  0.062263  
                ipw_ps            0.105401       0.089509  0.194910  
news-N_gaussian ipw_classifier    0.007800       0.003123  0.010922  
                ipw_hybrid        0.021437       0.025868  0.047306  
                ipw_ps            0.062239       0.071517  0.133756  
sim-B_gaussian  ipw_classifier    0.103754       0.026456  0.130209  
                ipw_hybrid        0.084724       0.118838  0.203561  
                ipw_ps            0.224828       0.300881  0.525708  
tcga-B_gaussian ipw_classifier    0.005370       0.070330  0.075700  
                ipw_hybrid        0.016472       0.055602  0.072073  
                ipw_ps            0.032621       0.069835  0.102455